## LangChain: Models, Prompts and Output Parsers
### Outline
* Direct API calls to OpenAI
* API calls through LangChain
    * Prompts
    * Models
    * Output Parsers

### Setup
1. Install the OpenAI & dotenv Python package as follows
```shell
$> pip install openai dotenv
```
2. Create an OpenAI API key from the [OpenAI API Website](https://openai.com/blog/openai-api)
3. Copy a new key and save it to a `.env` file as `OPENAI_API_KEY=sk-....` (replace `sk-....` with your key)

Load the API key and relevant Python libaries as shown below.

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# account for deprecation of LLM model
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

print(f"Will use LLM Model {llm_model}")

Will use LLM Model gpt-3.5-turbo


In [3]:
# create my chat client
from openai import OpenAI

CHAT_CLIENT = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)
MODEL = "gpt-3.5-turbo"  # enough for this tutorial

### Chat API: OpenAI
Let's start with direct calls to OpenAI.

In [4]:
# def get_completion(prompt, model=llm_model):
#     messages = [{"role": "user", "content": prompt}]
#     response = openai.ChatCompletion.create(
#         model=model,
#         messages=messages,
#         temperature=0,
#     )
#     return response.choices[0].message["content"]


def get_completion(
    user_prompt: str, client=CHAT_CLIENT, model=MODEL, get_tokens=False
) -> str:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
        model=model,
    )

    if get_tokens:
        # get tokens also
        tokens = {
            "completion_tokens": response.usage.completion_tokens,
            "prompt_tokens": response.usage.prompt_tokens,
            "total_tokens": response.usage.total_tokens,
        }
        return response.choices[0].message, tokens
    else:
        return response.choices[0].message

In [5]:
print(get_completion("What is the future of Generative AI?"))

ChatCompletionMessage(content='The future of Generative AI is promising, as this technology continues to advance and evolve. Some potential developments that could shape the future of Generative AI include:\n\n1. Better understanding and control over the output: As Generative AI models become more sophisticated, researchers are working on ways to better understand and control the outputs generated by these algorithms. This includes developing techniques to improve the quality, diversity, and coherence of the generated content.\n\n2. Human-AI collaboration: In the future, we may see increased collaboration between humans and Generative AI systems. This could involve using AI to assist with creative tasks, such as generating ideas or providing inspiration, while humans provide feedback and direction to help refine the final output.\n\n3. Personalized content generation: Generative AI could be used to create personalized content tailored to individual preferences and needs. This could inc

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [7]:
style = """American English \
in a calm and respectful tone
"""

In [8]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [9]:
response = get_completion(prompt)
print(response)

ChatCompletionMessage(content="I am frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I need your help immediately, my friend.", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


Now suppose you are a customer support rep, who receives emails from customers in different languages, like French, Spanish etc. You'd like to first translate it to "American English in a calm and respectful tone" so you can process it further.

While we will not cover this here, you may also want to type in a response to the email and have the LLM translate it back to the original language when responding to the customer.

In [10]:
customer_email_french = """
Qu'est-ce que c'est que Colorado Retail ? J'avais commandé un MacBook M3 pro 
avec 16 Go de RAM et 512 Go de SSD mais on m'a livré un MacBook Air M2 avec 8 Go 
de RAM et 256 SSD !! Et en plus vous m'avez facturé le M3 Pro et même Microsoft Office manque.  
J'attends un échange immédiat pour le produit que j'ai commandé!!!
"""

In [11]:
prompt2 = f"""Translate the text 
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email_french}```
"""

print(prompt2)

Translate the text 
that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Qu'est-ce que c'est que Colorado Retail ? J'avais commandé un MacBook M3 pro 
avec 16 Go de RAM et 512 Go de SSD mais on m'a livré un MacBook Air M2 avec 8 Go 
de RAM et 256 SSD !! Et en plus vous m'avez facturé le M3 Pro et même Microsoft Office manque.  
J'attends un échange immédiat pour le produit que j'ai commandé!!!
```



In [12]:
response = get_completion(prompt2)
print(response)

ChatCompletionMessage(content='What is Colorado Retail? I ordered a MacBook M3 Pro with 16GB of RAM and 512GB of SSD but I was delivered a MacBook Air M2 with 8GB of RAM and 256GB of SSD!! And on top of that, I was charged for the M3 Pro and even Microsoft Office is missing. I expect an immediate exchange for the product I ordered!!!', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


## Chat API: LangChain
Let's see how we can do the same in LangChain.

In [29]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

In [30]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(
    temperature=0.0,
    model_name=MODEL,
)

In [31]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [33]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'), additional_kwargs={})])

In [34]:
prompt_template.input_variables

['style', 'text']

In [35]:
customer_style = """American English \
in a calm and respectful tone
"""

In [36]:
customer_email_french = """
Qu'est-ce que c'est que Colorado Retail ? J'avais commandé un MacBook M3 pro 
avec 16 Go de RAM et 512 Go de SSD mais on m'a livré un MacBook Air M2 avec 8 Go 
de RAM et 256 SSD !! Et en plus vous m'avez facturé le M3 Pro et même Microsoft Office manque.  
J'attends un échange immédiat pour le produit que j'ai commandé!!!
"""

In [37]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email_french,
)

In [38]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [39]:
print(customer_messages[0])

content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nQu'est-ce que c'est que Colorado Retail ? J'avais commandé un MacBook M3 pro \navec 16 Go de RAM et 512 Go de SSD mais on m'a livré un MacBook Air M2 avec 8 Go \nde RAM et 256 SSD !! Et en plus vous m'avez facturé le M3 Pro et même Microsoft Office manque.  \nJ'attends un échange immédiat pour le produit que j'ai commandé!!!\n```\n" additional_kwargs={} response_metadata={}


In [42]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [43]:
print(customer_response.content)  # this should return the same response as before!

What is Colorado Retail? I ordered a MacBook M3 Pro with 16GB of RAM and 512GB of SSD, but I was delivered a MacBook Air M2 with 8GB of RAM and 256GB SSD!! And on top of that, I was charged for the M3 Pro and even Microsoft Office is missing. I expect an immediate exchange for the product I ordered!!!


In [44]:
service_reply = """Hey there customer, \
We are really sorry to hear that we delivered \
the wrong device to you. We are looking into \
the matter and please rest assured that we will \
rectify the mistake at the earliest. \
Thank you for your loyal patronage, and we \
hope you will continue to shop with Colorado Retail \
for all your needs. \
"""

In [45]:
service_style_french = """\
a polite tone \
that speaks in French\
"""

In [47]:
# We just call the LangChain API
service_messages = prompt_template.format_messages(
    style=service_style_french, text=service_reply
)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in French. text: ```Hey there customer, We are really sorry to hear that we delivered the wrong device to you. We are looking into the matter and please rest assured that we will rectify the mistake at the earliest. Thank you for your loyal patronage, and we hope you will continue to shop with Colorado Retail for all your needs. ```



In [48]:
service_response = chat.invoke(service_messages)
print(service_response.content)

Bonjour cher client,

Nous sommes vraiment désolés d'apprendre que nous vous avons livré le mauvais appareil. Nous examinons la situation et vous pouvez être assuré que nous rectifierons l'erreur dès que possible. Merci pour votre fidélité, et nous espérons que vous continuerez à faire vos achats chez Colorado Retail pour tous vos besoins.

Cordialement,


## Using Output Parsers

This is an example of using a JSON output parser

In [49]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [50]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [55]:
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


In [60]:
# NOTE: response.content is a STRING, not a dict, though it looks like one
# Now let us use output parsers to get true JSON
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [61]:
# now tell the parser what you want parsed out from output
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased\
        as a gift for someone else? \
        Answer True if yes,\
        False if not or unknown.",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days\
        did it take for the product\
        to arrive? If this \
        information is not found,\
        output -1.",
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any\
        sentences about the value or \
        price, and output them as a \
        comma separated Python list.",
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [62]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [63]:
# show me what instructions LangChain generated
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased        as a gift for someone else?         Answer True if yes,        False if not or unknown.
	"delivery_days": string  // How many days        did it take for the product        to arrive? If this         information is not found,        output -1.
	"price_value": string  // Extract any        sentences about the value or         price, and output them as a         comma separated Python list.
}
```


In [64]:
# now let's try it out
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(
    text=customer_review, format_instructions=format_instructions
)

In [65]:
print(messages)

[HumanMessage(content='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\ntext: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife\'s anniversary present. I think my wife liked it so much she was speechless. So far I\'ve been the only one using it, and I\'ve been using it every other morning to clear the leaves on our lawn. It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features.\n\n\nThe output should be a markdown code snippet formatted in the following schema, including the 

In [68]:
response = chat.invoke(messages)
print(type(response.content), "\n", response.content)  # still a string!

<class 'str'> 
 ```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [69]:
# now it converts to a JSON
output_dict = output_parser.parse(response.content)
print(type(output_dict), "\n", output_dict)

<class 'dict'> 
 {'gift': True, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


In [70]:
output_dict["gift"]

True